# Calculating stand level measurements from tree data

For this example, we'll start with loading some tree data and then calculate stand trees per acre, basal area. 

In [1]:
# Load packages
# Pkg.add("ForestBiometrics) # if install needed
using ForestBiometrics, CSV, DataFrames

#Load data
datapath = Pkg.dir("ForestBiometrics", "test", "data")
df=CSV.read(joinpath(datapath, "StandExam_data.csv"),nullable=false)


,Stand_ID,Plot_ID,Tree_ID,Tree_Count,History,Species,DBH,Ht
1,12345,1,1,1,1,WO,11,63
2,12345,1,2,1,1,WO,12,55
3,12345,1,3,1,1,WO,12,60
4,12345,1,4,1,1,WO,12,45
5,12345,1,5,1,1,WO,17,55
6,12345,1,6,1,1,WO,18,60
7,12345,1,7,1,1,WO,32,60
8,12345,1,8,1,1,WO,15,55
9,12345,1,9,1,1,WO,16,68
10,12345,1,10,1,1,RO,10,65


This is the tree data for a single stand with 17 plots taken. To simplify, a bit we'll just explain the specs here and not have to worry about a second table with cruise spec info. The plots were variable radius, measured with a BAF 10 prism.

ForestBiometrics exports a CONST ```K``` for the conversion factor between diameter in inches and area of a circle, as well as the metric version ```KMETRIC``` for diameter in centimeters.


In [2]:
K

S}, Base.Nullable{T}) in module Base at nullable.jl:238 overwritten in module NullableArrays at C:\Users\Casey\.julia\v0.6\NullableArrays\src\operators.jl:99.


0.005454154

In [3]:
KMETRIC

7.854e-5

# Basal Area

First let's start with calculating the basal area in ft^2 for each plot, so we need to get numbers of records/plot * 10 to get BA represented by that plot. Then we can generate our stand estimate for basal area/acre by taking the average of all plots in the stand.

In [4]:
#make some variables, so we can reference it for simplicity
BAF = 10
n_plots=17

plot_ba=by(df,:Plot_ID,df -> DataFrame(BA = size(df, 1)*BAF))



,Plot_ID,BA
1,1,100
2,2,80
3,3,90
4,4,100
5,5,100
6,6,100
7,7,110
8,8,70
9,9,80
10,10,110


In [5]:
stand_ba=mean(plot_ba[:BA])

90.58823529411765

# Trees per acre

TPA for variable radius plots is a little more involved, but nothing we can't handle. 
First, we'll calculate the appropriate expansion factor for each tree record and them combine them to get our estimate for the stand.

In [6]:
#Julia uses dots before operators (./, .+,.^) to apply functions element-wise in an array. 

#calculate basal area of each tree record
df[:TreeBA]=(df[:DBH].^2)*K

#calculating expansion factor for each tree record
df[:TreeEXP]=BAF./df[:TreeBA]./n_plots

# sum together to get stand estimate 
stand_tpa=sum(df[:TreeEXP])


147.85936586694856

Since, we are dealing with upland oak species, let's see where this stand falls on a gingrich chart. 

In [7]:
gingrich_chart(stand_tpa,stand_ba)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 350 
 
 
 400 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 140 
 
 
 Trees Per Acre 
 
 
 Basal Area (sq ft./acre) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20% 
 
 
 30% 
 
 
 40% 
 
 
 50% 
 
 
 60% 
 
 
 70% 
 
 
 80% 
 
 
 90% 
 
 
 100% 
 
 
 110%

It looks like the stand is appropriately stocked, sitting just below 80%. Let's see what the QMD of the stand is.

In [8]:
#this function will be added to the next release, but I forgot to add it in previously.
function qmd(ba, tpa, constant)
    qmd=sqrt((ba/tpa)/constant)
end

stand_qmd=qmd(stand_ba,stand_tpa,K)

10.598582432435709

Since the stand is not fully competing yet and QMD is just starting to get into sawtimber range, a landowner may decide to let this stand continue to develop for a few more years before re-evaluation for harvest or thinning. 

In this tutorial we have seen how we can work some stand level info up and start to use that information to make decisions. As the package develops, I hope that this can then blend other higher level operations such as growth and yield. 